# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [1]:
import gym
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

from collections import deque  # for memory
from tqdm import tqdm          # for progress bar

import numpy as np

In [2]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

How the game looks (without our agent):

In [3]:
'''
env = gym.make('CartPole-v1', render_mode='human')
for _ in tqdm(range(10)):
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
env.close()
'''

"\nenv = gym.make('CartPole-v1', render_mode='human')\nfor _ in tqdm(range(10)):\n    state, _ = env.reset()\n    done = False\n    while not done:\n        action = env.action_space.sample()\n        next_state, reward, done, _, _ = env.step(action)\nenv.close()\n"

## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

$\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}$

However, we don't know $Q^*$. So, we use neural network as a approximators, we can simply create one and train it to resemble $Q^*$.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

$\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}$

The difference between the two sides of the equality is known as the temporal difference error, $\delta$:

$\begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align}$

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.

In [10]:
class Model(nn.Module):
    def __init__(self, observation_size, action_size):
        super(Model, self).__init__()
        # initialise layers here
        self.dense1 = nn.Linear(observation_size, 100)
        torch.nn.init.xavier_uniform_(self.dense1.weight)
        self.dense2 = nn.Linear(100, 100)
        torch.nn.init.xavier_uniform_(self.dense2.weight)
        self.dense3 = nn.Linear(100, 100)
        torch.nn.init.xavier_uniform_(self.dense3.weight)
        self.dense4 = nn.Linear(100, action_size)
        torch.nn.init.xavier_uniform_(self.dense4.weight)

    def forward(self, x):
        # send x through the network
        x = self.dense1(x)
        x = F.relu(x)
        x = self.dense2(x)
        x = F.relu(x)
        x = self.dense3(x)
        x = F.relu(x)
        x = self.dense4(x)
        return x

    def predict(self, x):
        #x = torch.Tensor(x)
        x = self.forward(x)
        return x

### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [11]:
env = gym.make('CartPole-v1')
#env.reset()

In [12]:
from logging import critical


class Agent:
    def __init__(self, observation_size, action_size):
        self.observation_size = observation_size
        self.action_size = action_size
        self.criterion = nn.MSELoss()
        self.model = Model(observation_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.discount_value = 0.9
        self.epsilon = 0.15
        self.epsilondecay = 0.995
        #When appending to deque, if we exceed the maximum size we remove the oldest element of the list
        self.memory = deque([], maxlen=50000) # memory that stores N most new transitions
        # good place to store hyperparameters as attributes
    def remember(self, state, action, reward, next_state, done):
        # add to memory
        self.memory.append([state, action, reward, next_state, done])

    def act(self, state):
        # return an action from the model
        #print (self.epsilon)
        if random.uniform(0,1) < self.epsilon:
            action = env.action_space.sample()
            action = int(action)
            return action
        else:
            action = torch.argmax(self.model.predict(torch.tensor(state)))
            action = int(action)
            return action

        #pass

    def replay(self, batch_size):
        # update model based on replay memory
        # you might want to make a self.train() helper method
        
        minibatch = random.sample(self.memory, batch_size)
        self.optimizer.zero_grad()
        for i in range(batch_size):
            self.train(minibatch[i])
            
        self.epsilon *= self.epsilondecay
        self.optimizer.step()        

    def train(self, sample):
        s, a, r, st1, d = sample
        r = torch.tensor(r)
        if not d:
            value = r + self.discount_value * float(torch.max(self.model.forward(torch.tensor(st1))))
        else:
            value = r
        pred = self.model.forward(torch.tensor(s))[a]
        loss = self.criterion(pred, value)
        loss.backward()
        return int(loss)


### Main Training loop
---
Make a function that takes and environment and an agent, and runs through $n$ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [8]:
def train(env, agent, episodes=1000, batch_size=64):  # train for many games
    best_reward = 0
    for _ in tqdm(range(episodes)):
        state, _ = env.reset()
        done = False
        total_reward = 0
        actionLeft = 0
        actionRight = 1
        while not done:
            # 1. make a move in game.
            action = agent.act(state)
            # 2. have the agent remember stuff.
            #action = env.action_space.sample()
            
            new_state, reward, done, _, _ = env.step(action)
            
            total_reward += reward
            agent.remember(state, action, reward, new_state, done)
            # 3. update state
            state = new_state
            # 4. if we have enough experiences in out memory, learn from a batch with replay.
            if len(agent.memory) >= batch_size:
                agent.replay(batch_size)
        #print ("Total reward for episode = " + str(total_reward) + "     ActionsLeft = " + str(actionLeft) + "      actionRight = " + str(actionRight))
        print ("Total reward = " + str(total_reward))
        if total_reward >= best_reward:
            best_reward = total_reward
        print ("Best reward found = " + str(best_reward))
    env.close()

### Putting it together
---
We train an agent on the environment:


In [18]:
env = gym.make('CartPole-v1', render_mode = 'human')
agent = Agent(env.observation_space.shape[0], env.action_space.n)
train(env, agent)
env.close()
torch.save(agent.model.state_dict(), 'model.pth')

agent.model.state_dict_(torch.load('model.pth'))

env = gym.make('CartPole-v1', render_mode ='human')
for _ in tqdm(range(10)):
        state, _ = env.reset()
        done = False
        while not done:
            # 1. make a move in game.
            action = agent.act(state)
            # 2. have the agent remember stuff
            state, reward, done, _, _ = env.step(action)
            # 3. update state
env.close()


  0%|          | 1/1000 [00:00<08:13,  2.02it/s]

Total reward = 9.0
Best reward found = 9.0


  0%|          | 2/1000 [00:00<05:45,  2.89it/s]

Total reward = 11.0
Best reward found = 11.0


  0%|          | 3/1000 [00:00<04:59,  3.33it/s]

...

Total reward = 156.0
Best reward found = 156.0


  2%|▏         | 24/1000 [01:03<1:36:51,  5.95s/it]

Total reward = 186.0
Best reward found = 186.0


  2%|▏         | 24/1000 [01:09<46:50,  2.88s/it]  


KeyboardInterrupt: 

In [19]:

agent.model.load_state_dict(torch.load('model.pth'))
env = gym.make('CartPole-v1', render_mode ='human')
for _ in tqdm(range(10)):
        state, _ = env.reset()
        done = False
        while not done:
            # 1. make a move in game.
            action = agent.act(state)
            # 2. have the agent remember stuff
            state, reward, done, _, _ = env.step(action)
            # 3. update state
env.close()

  0%|          | 0/10 [00:54<?, ?it/s]


KeyboardInterrupt: 

: 

## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/